# 論文 3：理解 LSTM 網路（Understanding LSTM Networks）
## Christopher Olah

### 實作 LSTM 及閘門視覺化

LSTM（長短期記憶）網路透過閘門式記憶單元解決梯度消失問題。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## LSTM 單元實作

LSTM 有三個閘門：
1. **遺忘閘門（Forget Gate）**：決定從細胞狀態中遺忘什麼
2. **輸入閘門（Input Gate）**：決定添加什麼新資訊
3. **輸出閘門（Output Gate）**：決定根據細胞狀態輸出什麼

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class LSTMCell:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # 為了效率將權重串接：[輸入; 隱藏狀態] -> 閘門
        concat_size = input_size + hidden_size
        
        # 遺忘閘門
        self.Wf = np.random.randn(hidden_size, concat_size) * 0.01
        self.bf = np.zeros((hidden_size, 1))
        
        # 輸入閘門
        self.Wi = np.random.randn(hidden_size, concat_size) * 0.01
        self.bi = np.zeros((hidden_size, 1))
        
        # 候選細胞狀態
        self.Wc = np.random.randn(hidden_size, concat_size) * 0.01
        self.bc = np.zeros((hidden_size, 1))
        
        # 輸出閘門
        self.Wo = np.random.randn(hidden_size, concat_size) * 0.01
        self.bo = np.zeros((hidden_size, 1))
    
    def forward(self, x, h_prev, c_prev):
        """
        LSTM 單元的前向傳遞
        
        x: 輸入 (input_size, 1)
        h_prev: 前一個隱藏狀態 (hidden_size, 1)
        c_prev: 前一個細胞狀態 (hidden_size, 1)
        
        回傳：
        h_next: 下一個隱藏狀態
        c_next: 下一個細胞狀態
        cache: 反向傳遞需要的值
        """
        # 串接輸入和前一個隱藏狀態
        concat = np.vstack([x, h_prev])
        
        # 遺忘閘門：決定從細胞狀態中遺忘什麼
        f = sigmoid(np.dot(self.Wf, concat) + self.bf)
        
        # 輸入閘門：決定儲存什麼新資訊
        i = sigmoid(np.dot(self.Wi, concat) + self.bi)
        
        # 候選細胞狀態：可能要添加的新資訊
        c_tilde = np.tanh(np.dot(self.Wc, concat) + self.bc)
        
        # 更新細胞狀態：遺忘 + 輸入新資訊
        c_next = f * c_prev + i * c_tilde
        
        # 輸出閘門：決定輸出什麼
        o = sigmoid(np.dot(self.Wo, concat) + self.bo)
        
        # 隱藏狀態：經過過濾的細胞狀態
        h_next = o * np.tanh(c_next)
        
        # 快取以供反向傳遞使用
        cache = (x, h_prev, c_prev, concat, f, i, c_tilde, c_next, o, h_next)
        
        return h_next, c_next, cache

# 測試 LSTM 單元
input_size = 10
hidden_size = 20
lstm_cell = LSTMCell(input_size, hidden_size)

x = np.random.randn(input_size, 1)
h = np.zeros((hidden_size, 1))
c = np.zeros((hidden_size, 1))

h_next, c_next, cache = lstm_cell.forward(x, h, c)
print(f"LSTM 單元已初始化：input_size={input_size}, hidden_size={hidden_size}")
print(f"隱藏狀態形狀：{h_next.shape}")
print(f"細胞狀態形狀：{c_next.shape}")

## 用於序列處理的完整 LSTM 網路

In [ ]:
class LSTM:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        self.cell = LSTMCell(input_size, hidden_size)
        
        # 輸出層
        self.Why = np.random.randn(output_size, hidden_size) * 0.01
        self.by = np.zeros((output_size, 1))
    
    def forward(self, inputs):
        """
        通過 LSTM 處理序列
        inputs: 輸入向量的列表
        """
        h = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))
        
        # 儲存狀態以供視覺化
        h_states = []
        c_states = []
        gate_values = {'f': [], 'i': [], 'o': []}
        
        for x in inputs:
            h, c, cache = self.cell.forward(x, h, c)
            h_states.append(h.copy())
            c_states.append(c.copy())
            
            # 從快取中提取閘門值
            _, _, _, _, f, i, _, _, o, _ = cache
            gate_values['f'].append(f.copy())
            gate_values['i'].append(i.copy())
            gate_values['o'].append(o.copy())
        
        # 最終輸出
        y = np.dot(self.Why, h) + self.by
        
        return y, h_states, c_states, gate_values

# 建立 LSTM 模型
input_size = 5
hidden_size = 16
output_size = 5
lstm = LSTM(input_size, hidden_size, output_size)
print(f"\nLSTM 模型已建立：{input_size} -> {hidden_size} -> {output_size}")

## 合成序列任務測試：長期依賴

任務：記住序列開頭的值，並在結尾輸出它

In [ ]:
def generate_long_term_dependency_data(seq_length=20, num_samples=100):
    """
    生成需要記住第一個元素直到最後的序列
    """
    X = []
    y = []
    
    for _ in range(num_samples):
        # 建立序列
        sequence = []
        
        # 第一個元素是重要的（one-hot 編碼）
        first_elem = np.random.randint(0, input_size)
        first_vec = np.zeros((input_size, 1))
        first_vec[first_elem] = 1
        sequence.append(first_vec)
        
        # 其餘是隨機雜訊
        for _ in range(seq_length - 1):
            noise = np.random.randn(input_size, 1) * 0.1
            sequence.append(noise)
        
        X.append(sequence)
        
        # 目標：記住第一個元素
        target = np.zeros((output_size, 1))
        target[first_elem] = 1
        y.append(target)
    
    return X, y

# 生成測試資料
X_test, y_test = generate_long_term_dependency_data(seq_length=15, num_samples=10)

# 測試前向傳遞
output, h_states, c_states, gate_values = lstm.forward(X_test[0])

print(f"\n測試序列長度：{len(X_test[0])}")
print(f"第一個元素（需記住）：{np.argmax(X_test[0][0])}")
print(f"預期輸出：{np.argmax(y_test[0])}")
print(f"模型輸出（未訓練）：{output.flatten()[:5]}")

## 視覺化 LSTM 閘門

理解 LSTM 的關鍵是觀察閘門如何隨時間運作。

In [ ]:
# 處理序列並視覺化閘門
test_seq = X_test[0]
output, h_states, c_states, gate_values = lstm.forward(test_seq)

# 轉換為陣列以供繪圖
forget_gates = np.hstack(gate_values['f'])
input_gates = np.hstack(gate_values['i'])
output_gates = np.hstack(gate_values['o'])
cell_states = np.hstack(c_states)
hidden_states = np.hstack(h_states)

fig, axes = plt.subplots(5, 1, figsize=(14, 12))

# 遺忘閘門
axes[0].imshow(forget_gates, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
axes[0].set_title('遺忘閘門（1=保留，0=遺忘）')
axes[0].set_ylabel('隱藏單元')
axes[0].set_xlabel('時間步')

# 輸入閘門
axes[1].imshow(input_gates, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
axes[1].set_title('輸入閘門（1=接受新資訊，0=忽略新資訊）')
axes[1].set_ylabel('隱藏單元')
axes[1].set_xlabel('時間步')

# 輸出閘門
axes[2].imshow(output_gates, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
axes[2].set_title('輸出閘門（1=暴露，0=隱藏）')
axes[2].set_ylabel('隱藏單元')
axes[2].set_xlabel('時間步')

# 細胞狀態
im3 = axes[3].imshow(cell_states, cmap='RdBu', aspect='auto')
axes[3].set_title('細胞狀態（長期記憶）')
axes[3].set_ylabel('隱藏單元')
axes[3].set_xlabel('時間步')
plt.colorbar(im3, ax=axes[3])

# 隱藏狀態
im4 = axes[4].imshow(hidden_states, cmap='RdBu', aspect='auto')
axes[4].set_title('隱藏狀態（輸出到下一層）')
axes[4].set_ylabel('隱藏單元')
axes[4].set_xlabel('時間步')
plt.colorbar(im4, ax=axes[4])

plt.tight_layout()
plt.show()

print("\n閘門解釋：")
print("- 遺忘閘門控制從細胞狀態中丟棄什麼資訊")
print("- 輸入閘門控制向細胞狀態添加什麼新資訊")
print("- 輸出閘門控制從細胞狀態輸出什麼")
print("- 細胞狀態是長期記憶的高速公路")

## 比較 LSTM 與原始 RNN 在長序列上的表現

In [ ]:
class VanillaRNNCell:
    def __init__(self, input_size, hidden_size):
        concat_size = input_size + hidden_size
        self.Wh = np.random.randn(hidden_size, concat_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.hidden_size = hidden_size
    
    def forward(self, x, h_prev):
        concat = np.vstack([x, h_prev])
        h_next = np.tanh(np.dot(self.Wh, concat) + self.bh)
        return h_next

# 建立原始 RNN 作為比較
rnn_cell = VanillaRNNCell(input_size, hidden_size)

def process_with_vanilla_rnn(inputs):
    h = np.zeros((hidden_size, 1))
    h_states = []
    
    for x in inputs:
        h = rnn_cell.forward(x, h)
        h_states.append(h.copy())
    
    return h_states

# 用兩種方法處理相同序列
rnn_h_states = process_with_vanilla_rnn(test_seq)
rnn_hidden = np.hstack(rnn_h_states)

# 比較隱藏狀態演化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

im1 = ax1.imshow(rnn_hidden, cmap='RdBu', aspect='auto')
ax1.set_title('原始 RNN 隱藏狀態')
ax1.set_ylabel('隱藏單元')
ax1.set_xlabel('時間步')
plt.colorbar(im1, ax=ax1)

im2 = ax2.imshow(hidden_states, cmap='RdBu', aspect='auto')
ax2.set_title('LSTM 隱藏狀態')
ax2.set_ylabel('隱藏單元')
ax2.set_xlabel('時間步')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

print("\n關鍵差異：")
print("- LSTM 維護與隱藏狀態分開的細胞狀態")
print("- 閘門允許選擇性的資訊流動")
print("- 更好的時間梯度流動（解決梯度消失問題）")

## 梯度流動比較

In [ ]:
# 模擬梯度大小
def simulate_gradient_flow(seq_length=30):
    """
    模擬原始 RNN 與 LSTM 中梯度如何衰減
    """
    # 原始 RNN：梯度指數衰減
    rnn_grads = []
    grad = 1.0
    decay_factor = 0.85  # 原始 RNN 的典型衰減
    
    for t in range(seq_length):
        rnn_grads.append(grad)
        grad *= decay_factor
    
    # LSTM：梯度通過細胞狀態高速公路保持
    lstm_grads = []
    grad = 1.0
    forget_gate_avg = 0.95  # 高遺忘閘門值 = 保持梯度
    
    for t in range(seq_length):
        lstm_grads.append(grad)
        grad *= forget_gate_avg  # 遺忘閘門控制梯度流動
    
    return np.array(rnn_grads), np.array(lstm_grads)

rnn_grads, lstm_grads = simulate_gradient_flow()

plt.figure(figsize=(12, 5))
plt.plot(rnn_grads[::-1], label='原始 RNN', linewidth=2)
plt.plot(lstm_grads[::-1], label='LSTM', linewidth=2)
plt.xlabel('過去的時間步數')
plt.ylabel('梯度大小')
plt.title('梯度流動：LSTM vs 原始 RNN')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\n30 步後的梯度：")
print(f"原始 RNN：{rnn_grads[-1]:.6f}（已消失）")
print(f"LSTM：{lstm_grads[-1]:.6f}（已保持）")
print(f"\n這就是為什麼 LSTM 能夠學習長期依賴！")

## 關鍵要點

### LSTM 架構：
1. **細胞狀態**：資訊跨時間流動的高速公路
2. **遺忘閘門**：控制從記憶中移除什麼
3. **輸入閘門**：控制添加什麼新資訊
4. **輸出閘門**：控制從記憶中輸出什麼

### 為什麼 LSTM 有效：
- **恆定誤差傳送帶（Constant Error Carousel）**：細胞狀態提供不間斷的梯度流動
- **乘法閘門**：讓網路學習何時記住/遺忘
- **加法更新**：細胞狀態通過加法更新 (f*c + i*c_tilde)
- **梯度保持**：遺忘閘門接近 1 時保持梯度

### 相對於原始 RNN 的優勢：
- 解決梯度消失問題
- 學習長期依賴（100+ 時間步）
- 更穩定的訓練
- 在實際序列任務中表現更好